In [3]:

import pandas as pd
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde


import warnings
warnings.filterwarnings('ignore')


In [4]:
## RUTA BASES
# ruta_input = r'C:\Users\User_1\Desktop\PUCP\APOYO\02 Proyectos\Elecciones_2021\inputs' <- EJEMPLO
# ruta_bases = r'C:\Users\User_1\Desktop\PUCP\APOYO\02 Proyectos\Elecciones_2021\inputs\hoja_vida' <- EJEMPLO
# ruta_insumos = r'C:\Users\User_1\Desktop\PUCP\APOYO\02 Proyectos\Elecciones_2021\insumos_bi' <- EJEMPLO

ruta_input = r'<:ingresa una ruta correspondiente a los archivos:>'
ruta_bases = r'<:ingresa una ruta correspondiente a los archivos:>'
ruta_insumos = r'<:ingresa una ruta correspondiente a los archivos:>'


### IMPORTACIÓN DE BASES

In [5]:
## INFO GENERAL DE PARTIDOS Y CANDIDATOS CONGRESALES
df_partidos_cong = pd.read_csv(ruta_input + '/base_partidos_congresal.csv',encoding='latin-1',
                         dtype='str')
df_candidatos_cong = pd.read_csv(ruta_input + '/base_candidatos_congresal.csv',encoding='latin-1',
                            dtype='str')

df_part_cand_cong = df_partidos_cong.merge(df_candidatos_cong,how='left',on='idSolicitudLista',suffixes=[None,'_cand'])

## UBIGEO DEPARTAMENTOS DE POSTULACIÓN
df_departamentos = pd.read_csv(ruta_input + '/base_departamentos.csv',encoding='latin-1',dtype={'ubigeo_region':'str'})

df_part_cand_cong = df_part_cand_cong.merge(df_departamentos,how='left',right_on='ubigeo_region',left_on='strUbigeo')


In [45]:
## FUNCION: IMPORTACIÓN DE BASES HOJAS DE VIDA

def import_base_hdv(ruta_excel,merge_key='idHojaVida'):
    df_base = pd.read_excel(ruta_bases + ruta_excel,encoding='utf-8-sig',dtype='str')
    
    ## Añadimos info extra
    df_base = df_base.merge(df_part_cand_cong[[merge_key,'strOrganizacionPolitica','strCandidato','region']],
                                          on=merge_key,how='left')
    
    ## Filtrado de solo congresistas
    df_base = df_base[df_base[merge_key].isin(df_part_cand_cong[merge_key])]
    
    ## Añadimos una región [GENERAL]
    
#     df_base_general = df_base.copy()
#     df_base_general['region'] = 'GENERAL'
    
#     df_base = pd.concat([df_base,df_base_general])
    
    return df_base


In [46]:
## IMPORTACIÓN HDV

df_info_personal = import_base_hdv('/base_candidatos_DatosPersonales.xlsx',merge_key='idCandidato')
df_ingresos = import_base_hdv('/base_candidatos_Ingresos.xlsx')

df_exp_laboral = import_base_hdv('/base_candidatos_ExperienciaLaboral.xlsx')

df_sentencia_oblig = import_base_hdv('/base_candidatos_SentenciaObliga.xlsx')
df_sentencia_penal = import_base_hdv('/base_candidatos_SentenciaPenal.xlsx')

df_exp_partidaria = import_base_hdv('/base_candidatos_CargoPartidario.xlsx')
df_exp_elecciones = import_base_hdv('/base_candidatos_CargoEleccion.xlsx')

df_edu_postgrado = import_base_hdv('/base_candidatos_EduPosgrago.xlsx')
df_edu_univ = import_base_hdv('/base_candidatos_EduUniversitaria_2.xlsx')

df_bienes_inmuebles = import_base_hdv('/base_candidatos_BienInmueble.xlsx')
df_bienes_muebles = import_base_hdv('/base_candidatos_BienMueble.xlsx')

## GENERACIÓN DE INSUMOS [1-?]

### 1. INGRESOS PROMEDIO

In [49]:
df_ingresos_declarados = df_ingresos[df_ingresos['strTengoIngresos']=='1']

df_ingresos_declarados['ingreso_publico'] = df_ingresos_declarados[['decRemuBrutaPublico','decRentaIndividualPublico',
                                                       'decOtroIngresoPublico']].astype('float').sum(axis=1)

df_ingresos_declarados['ingreso_privado'] = df_ingresos_declarados[['decRemuBrutaPrivado','decRentaIndividualPrivado',
                                                       'decOtroIngresoPrivado']].astype('float').sum(axis=1)

df_ingresos_declarados['ingreso_total'] = df_ingresos_declarados[['ingreso_publico',
                                                                   'ingreso_privado']].astype('float').sum(axis=1)

In [55]:
df_ingresos_cand = df_ingresos_declarados[['idHojaVida','ingreso_publico','ingreso_privado','ingreso_total']] \
                                        .drop_duplicates('idHojaVida')

### 2. EXPERIENCIA LABORAL

In [58]:
def get_año_exp_neto(x,col1,col2):
    str_list_1 = x[col1]
    str_list_2 = x[col2]
    
    list_1 = [int(x) for x in str_list_1.tolist()]
    list_2 = [int(x) for x in str_list_2.tolist()]
    list_2 = [datetime.now().year if x == 0 else x for x in list_2[:]]

    total_years = []
    for year_ini,year_fin in zip(list_1,list_2):
        if year_fin == 0: year_fin = datetime.now().year
        total_years += list(range(year_ini,year_fin+1))

    same_years_acum = 0
    for pair in list(set(zip(list_1,list_2))):
        if pair[0] == pair[1]: same_years_acum += 1
            
    total_years = list(set(total_years))
    total_years.sort()
    
    return get_years_acum(total_years,list_1,list_2) + same_years_acum


In [59]:
def get_years_acum(total_years,list_1,list_2):
    years_acum = 0
    for _index in range(0,len(total_years)):
        if _index == (len(total_years)-1): break
        signal=True
        
        if (total_years[_index+1] - total_years[_index]) == 1 :
            if total_years[_index] in list_2:
                if total_years[_index] in list_1:
                    for index_list in range(0,len(list_1)):
                        if (total_years[_index] == list_1[index_list]) and (list_2[index_list] != list_1[index_list]):
                            years_acum += 1
                            signal= False
                            break
                    if signal:
                        for x,y in zip(list_1,list_2):
                            if (total_years[_index] in range(x,y+1)) and (total_years[_index] != y):
                                years_acum += 1
                                break
                else:
                    for x,y in zip(list_1,list_2):
                        if (total_years[_index] in range(x,y+1)) and (total_years[_index] != y):
                            years_acum += 1
                            break
            else:
                years_acum += 1
                
    return years_acum

In [65]:
df_exp_laboral_declarados = df_exp_laboral[df_exp_laboral['strTengoExpeLaboral']=='1'].drop_duplicates(subset='idHVExpeLaboral')

df_exp_laboral_cand = df_exp_laboral[['idHojaVida','strCandidato']]

df_n_empleos = df_exp_laboral_declarados.groupby('idHojaVida').size().reset_index(name='n_empleos')

df_años_exp = df_exp_laboral_declarados.groupby('idHojaVida') \
                                        .apply(lambda x: get_año_exp_neto(x,'strAnioTrabajoDesde','strAnioTrabajoHasta')) \
                .reset_index(name='años_exp')


### 4. SENTENCIAS PENALES Y OBLIGATORIAS

In [91]:
## numero de sentencias penales
df_penal_cand = df_sentencia_penal[['idHojaVida','strTengoSentenciaPenal']] \
                                        .drop_duplicates('idHojaVida')

df_penal_cand['strTengoSentenciaPenal'] = df_penal_cand['strTengoSentenciaPenal'].apply(lambda x: '0' if x == '2' else '1')

## numero candidatos con sentencias obligatorias
df_oblig_cand = df_sentencia_oblig[['idHojaVida','strTengoSentenciaObliga']] \
                                        .drop_duplicates('idHojaVida')

df_oblig_cand['strTengoSentenciaObliga'] = df_oblig_cand['strTengoSentenciaObliga'].apply(lambda x: '0' if x == '2' else '1')


### 5. EXPERIENCIA PARTIDARIA

In [99]:
df_exp_partidaria_declarada = df_exp_partidaria[df_exp_partidaria['strTengoCargoPartidario']=='1']

df_años_exp_parti = df_exp_partidaria_declarada.groupby('idHojaVida') \
                        .apply(lambda x: get_año_exp_neto(x,'strAnioCargoPartiDesde','strAnioCargoPartiHasta')) \
                        .reset_index(name='años_exp_partidaria')


### 5.5 COHESIÓN PARTIDARIA

In [132]:
df_años_partido = df_exp_partidaria_declarada[~df_exp_partidaria_declarada['strOrgPolCargoPartidario'].isnull()]
df_años_partido = df_años_partido[df_años_partido.apply(
                                lambda x: (x['strOrganizacionPolitica'] in x['strOrgPolCargoPartidario']),axis=1)]

df_años_cohesion = df_años_partido.groupby('idHojaVida') \
                                        .apply(lambda x: get_año_exp_neto(x,'strAnioCargoPartiDesde','strAnioCargoPartiHasta')) \
                .reset_index(name='años_cohe_partido')


### 6. EXPERIENCIA POLITICA

In [104]:
df_exp_politica_declarada = df_exp_elecciones[df_exp_elecciones['strCargoEleccion']=='1']
df_exp_politica_declarada = df_exp_politica_declarada[~df_exp_politica_declarada['strAnioCargoElecDesde'].isnull()]

df_años_exp_politica = df_exp_politica_declarada.groupby('idHojaVida') \
                        .apply(lambda x: get_año_exp_neto(x,'strAnioCargoElecDesde','strAnioCargoElecHasta')) \
                        .reset_index(name='años_exp_politica')


### 7. EDUCACIÓN SUPERIOR

In [121]:
df_edu_univ_cand = df_edu_univ[['idHojaVida','strTengoEduUniversitaria']].drop_duplicates('idHojaVida')

df_edu_univ_cand['strTengoEduUniversitaria'] = df_edu_univ_cand['strTengoEduUniversitaria'].apply(lambda x: '0' if x == '2' else '1')

df_edu_post_cand = df_edu_postgrado[['idHojaVida','strConcluidoPosgrado','strEsMaestro','strEsDoctor']].drop_duplicates('idHojaVida')

df_edu_sup_cand = df_edu_univ_cand.merge(df_edu_post_cand,on='idHojaVida',how='left')

## CONSOLIDADO DE BASES

In [125]:
df_candidatos_base = df_part_cand_cong[['region','strOrganizacionPolitica','intPosicion','idHojaVida',
                                       'strCandidato','strRutaArchivo','intEdad',
                                        'strDocumentoIdentidad','strFechaNacimiento']]

df_candidatos_base['strFechaNacimiento'] = df_candidatos_base['strFechaNacimiento'].str[:10]
df_candidatos_base['strRutaArchivo'] = 'https://declara.jne.gob.pe' + df_candidatos_base['strRutaArchivo']


In [137]:
## INGRESOS
df_candidatos_2021 = df_candidatos_base.merge(df_ingresos_cand,how='left',on='idHojaVida')

## EXPERIENCIA LABORAL
df_candidatos_2021 = df_candidatos_2021.merge(df_n_empleos,how='left',on='idHojaVida')
df_candidatos_2021 = df_candidatos_2021.merge(df_años_exp,how='left',on='idHojaVida')

## SENTENCIAS PENALES Y OBLIGATORIAS
df_candidatos_2021 = df_candidatos_2021.merge(df_penal_cand,how='left',on='idHojaVida')
df_candidatos_2021 = df_candidatos_2021.merge(df_oblig_cand,how='left',on='idHojaVida')


## EXPERIENCIA PARTIDARIA (NETA Y DEL PARTIDO) Y POLITICA
df_candidatos_2021 = df_candidatos_2021.merge(df_años_exp_parti,how='left',on='idHojaVida')
df_candidatos_2021['años_exp_partidaria'] = df_candidatos_2021['años_exp_partidaria'].fillna(0)

df_candidatos_2021 = df_candidatos_2021.merge(df_años_cohesion,how='left',on='idHojaVida')
df_candidatos_2021['años_cohe_partido'] = df_candidatos_2021['años_cohe_partido'].fillna(0)

df_candidatos_2021 = df_candidatos_2021.merge(df_años_exp_politica,how='left',on='idHojaVida')
df_candidatos_2021['años_exp_politica'] = df_candidatos_2021['años_exp_politica'].fillna(0)

## EDUCACION SUPERIOR
df_candidatos_2021 = df_candidatos_2021.merge(df_edu_sup_cand,how='left',on='idHojaVida')


In [142]:
df_candidatos_2021

,region,strOrganizacionPolitica,intPosicion,idHojaVida,strCandidato,strRutaArchivo,intEdad,strDocumentoIdentidad,strFechaNacimiento,ingreso_publico,...,años_exp,strTengoSentenciaPenal,strTengoSentenciaObliga,años_exp_partidaria,años_cohe_partido,años_exp_politica,strTengoEduUniversitaria,strConcluidoPosgrado,strEsMaestro,strEsDoctor
0,PERUANOS RESIDENTES EN EL EXTRANJERO,RENACIMIENTO UNIDO NACIONAL,1,134806,FREDDY RAYMUNDO BOLAÑOS SOTOMAYOR,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,48,10167038,23/01/1972,0.0,...,10.0,0,0,0.0,0.0,0.0,1,0,0,0
1,PERUANOS RESIDENTES EN EL EXTRANJERO,RENACIMIENTO UNIDO NACIONAL,2,134404,VIOLETA GLORIA BONIFACIO TORREJON,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,42,10875009,06/08/1978,0.0,...,4.0,0,0,0.0,0.0,0.0,1,2,0,0
2,PERUANOS RESIDENTES EN EL EXTRANJERO,RENACIMIENTO UNIDO NACIONAL,3,134410,IRLANDO ALBORNOZ CIPIRIANO,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,31,46621505,23/12/1988,0.0,...,7.0,0,0,0.0,0.0,0.0,0,0,0,0
3,PERUANOS RESIDENTES EN EL EXTRANJERO,RENACIMIENTO UNIDO NACIONAL,4,134416,GRACIELA GOMEZ CHACON,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,47,24994968,12/06/1973,NaN,...,NaN,0,0,0.0,0.0,0.0,0,0,0,0
4,PERUANOS RESIDENTES EN EL EXTRANJERO,UNION POR EL PERU,1,137737,JORGE ANTONIO FARFAN GAVIRIA,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,59,06168750,05/05/1961,NaN,...,32.0,0,0,0.0,0.0,0.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3021,UCAYALI,JUNTOS POR EL PERU,4,136666,YARA MARINA RUIZ GUERRERO,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,60,00068891,25/04/1960,12696.0,...,7.0,0,0,0.0,0.0,0.0,1,0,0,0
3022,UCAYALI,UNION POR EL PERU,1,136312,WALTER ACHA ROMANI,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,53,28297823,31/10/1967,0.0,...,5.0,0,0,5.0,0.0,5.0,1,0,0,0
3023,UCAYALI,UNION POR EL PERU,2,136396,LUZ MARITZA CASSANA NAVARRO,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,52,10379271,28/05/1968,NaN,...,1.0,0,0,0.0,0.0,0.0,1,0,0,0
3024,UCAYALI,UNION POR EL PERU,3,136682,RICARDO SUSANIBAR GUARDAMINO,https://declara.jne.gob.pe/Assets/Fotos-HojaVi...,68,00078860,02/04/1952,NaN,...,11.0,1,0,0.0,0.0,0.0,1,0,0,0


In [146]:
df_candidatos_2021['url_hoja_vida'] = 'https://otorongo.club/2021/candidato/' + df_candidatos_2021['strDocumentoIdentidad'] + '/'


In [148]:
df_candidatos_2021.to_excel(ruta_insumos + '/insumos_candidatos v1.xlsx',index=False,encoding='utf-8-sig')